In [ ]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [2]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]


def main():
  """Shows basic usage of the Gmail API.
  Lists the user's Gmail labels.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("../secrets/token.json"):
    creds = Credentials.from_authorized_user_file("../secrets/token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "../secrets/credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("../secrets/token.json", "w") as token:
      token.write(creds.to_json())

  try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("No labels found.")
      return
    print("Labels:")
    for label in labels:
      print(label["name"])

  except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=69722697557-n13nrkrq7t630sthrt70fucdcmn19hsl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A55494%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=A0jvh4z1XQ3qAgBfHpMa6PvgZZ6Rz2&access_type=offline
Labels:
CHAT
SENT
INBOX
IMPORTANT
TRASH
DRAFT
SPAM
CATEGORY_FORUMS
CATEGORY_UPDATES
CATEGORY_PERSONAL
CATEGORY_PROMOTIONS
CATEGORY_SOCIAL
STARRED
UNREAD


---

In [18]:
import os.path
import base64
import mimetypes
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

from typing import Any, List, Dict, Optional

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Definimos los alcances necesarios para la API de Gmail
SCOPES = ['https://mail.google.com/']

class Gmail():
    @classmethod
    def get_credentials(cls, secrets_path: str):
        creds = None
        if os.path.exists(secrets_path + "token.json"):
            creds = Credentials.from_authorized_user_file(secrets_path + "token.json", SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    secrets_path + "credentials.json", SCOPES
                )
                creds = flow.run_local_server(port=0)
            with open(secrets_path + "token.json", "w") as token:
                token.write(creds.to_json())
        return creds

    @classmethod
    def get_service(cls, secrets_path: str = "../secrets/"):
        creds = cls.get_credentials(secrets_path)
        return build('gmail', 'v1', credentials=creds)

    @classmethod
    def create_message(cls, sender: str, to: str, subject: str, message_text: str) -> Dict[str, Any]:
        message = MIMEText(message_text)
        message['to'] = to
        message['from'] = sender
        message['subject'] = subject
        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        return {'raw': raw}

    @classmethod
    def create_message_with_attachment(cls, sender: str, to: str, subject: str, message_text: str, file_path: str) -> Dict[str, Any]:
        message = MIMEMultipart()
        message['to'] = to
        message['from'] = sender
        message['subject'] = subject

        message.attach(MIMEText(message_text))

        content_type, encoding = mimetypes.guess_type(file_path)
        if content_type is None or encoding is not None:
            content_type = 'application/octet-stream'

        main_type, sub_type = content_type.split('/', 1)
        with open(file_path, 'rb') as f:
            if main_type == 'text':
                attachment = MIMEText(f.read().decode('utf-8'), _subtype=sub_type)
            elif main_type == 'image':
                from email.mime.image import MIMEImage
                attachment = MIMEImage(f.read(), _subtype=sub_type)
            elif main_type == 'audio':
                from email.mime.audio import MIMEAudio
                attachment = MIMEAudio(f.read(), _subtype=sub_type)
            else:
                attachment = MIMEBase(main_type, sub_type)
                attachment.set_payload(f.read())
                encoders.encode_base64(attachment)

        filename = os.path.basename(file_path)
        attachment.add_header('Content-Disposition', 'attachment', filename=filename)
        message.attach(attachment)

        raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
        return {'raw': raw}

    @classmethod
    def send_message(cls, service: Any, user_id: str, message: Dict[str, Any]) -> Dict[str, Any]:
        try:
            sent_message = service.users().messages().send(userId=user_id, body=message).execute()
            return sent_message
        except HttpError as error:
            print(f'Ocurrió un error: {error}')
            return None

    @classmethod
    def list_messages(cls, service: Any, user_id: str = 'me', query: str = '') -> List[Dict[str, Any]]:
        try:
            response = service.users().messages().list(userId=user_id, q=query).execute()
            messages = response.get('messages', [])
            while 'nextPageToken' in response:
                page_token = response['nextPageToken']
                response = service.users().messages().list(userId=user_id, q=query, pageToken=page_token).execute()
                messages.extend(response.get('messages', []))
            return messages
        except HttpError as error:
            print(f'Ocurrió un error: {error}')
            return []

    @classmethod
    def get_message(cls, service: Any, message_id: str, user_id: str = 'me') -> Dict[str, Any]:
        try:
            message = service.users().messages().get(userId=user_id, id=message_id, format='full').execute()
            return message
        except HttpError as error:
            print(f'Ocurrió un error: {error}')
            return None

    @classmethod
    def delete_message(cls, service: Any, message_id: str, user_id: str = 'me') -> None:
        try:
            service.users().messages().delete(userId=user_id, id=message_id).execute()
        except HttpError as error:
            print(f'Ocurrió un error: {error}')

    @classmethod
    def modify_message(cls, service: Any, message_id: str, add_labels: List[str] = [], remove_labels: List[str] = [], user_id: str = 'me') -> Dict[str, Any]:
        try:
            message = service.users().messages().modify(
                userId=user_id,
                id=message_id,
                body={'addLabelIds': add_labels, 'removeLabelIds': remove_labels}
            ).execute()
            return message
        except HttpError as error:
            print(f'Ocurrió un error: {error}')
            return None

    @classmethod
    def create_draft(cls, service: Any, user_id: str, message_body: Dict[str, Any]) -> Dict[str, Any]:
        try:
            draft = service.users().drafts().create(userId=user_id, body={'message': message_body}).execute()
            return draft
        except HttpError as error:
            print(f'Ocurrió un error: {error}')
            return None

    @classmethod
    def send_draft(cls, service: Any, user_id: str, draft_id: str) -> Dict[str, Any]:
        try:
            sent_message = service.users().drafts().send(userId=user_id, body={'id': draft_id}).execute()
            return sent_message
        except HttpError as error:
            print(f'Ocurrió un error: {error}')
            return None

    @classmethod
    def list_labels(cls, service: Any, user_id: str = 'me') -> List[Dict[str, Any]]:
        try:
            response = service.users().labels().list(userId=user_id).execute()
            labels = response.get('labels', [])
            return labels
        except HttpError as error:
            print(f'Ocurrió un error: {error}')
            return []

    @classmethod
    def get_profile(cls, service: Any, user_id: str = 'me') -> Dict[str, Any]:
        try:
            profile = service.users().getProfile(userId=user_id).execute()
            return profile
        except HttpError as error:
            print(f'Ocurrió un error: {error}')
            return None

In [22]:
# Obtener el servicio de Gmail
service = Gmail.get_service(secrets_path='../secrets/')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=69722697557-n13nrkrq7t630sthrt70fucdcmn19hsl.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61424%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=sF5OBfwQDafLwqxYHzctVKg6TaD1Ue&access_type=offline


In [ ]:
# Crear el mensaje
message = Gmail.create_message(
    sender='tu_email@gmail.com',
    to='destinatario@example.com',
    subject='Asunto del correo',
    message_text='Este es el contenido del correo.'
)

# Enviar el mensaje
sent_message = Gmail.send_message(service, user_id='me', message=message)
print(f"Mensaje enviado. ID: {sent_message['id']}")

In [9]:
message_attended = Gmail.create_message_with_attachment(
    sender='tu_email@gmail.com',
    to='destinatario@example.com',
    subject='Asunto del correo',
    message_text='Este es el contenido del correo con un archivo.', 
    file_path='../main.py'
)

sent_message = Gmail.send_message(service, user_id='me', message=message_attended)
print(f"Mensaje enviado. ID: {sent_message['id']}")

Mensaje enviado. ID: 192d4f0a2c51f697


In [28]:
Gmail.list_messages(service, query='from:jorge.juarez@cua.uam.mx')

[{'id': '192b5091b9671c3f', 'threadId': '192b5091b9671c3f'},
 {'id': '18f2d783e2039482', 'threadId': '18f2d68e8657b295'},
 {'id': '18f2d741d8ff9729', 'threadId': '18f2d68e8657b295'},
 {'id': '18f2d73f1b12beec', 'threadId': '18f2d68e8657b295'},
 {'id': '18f2d6b4241f8d71', 'threadId': '18f2d68e8657b295'},
 {'id': '18f2d68e8657b295', 'threadId': '18f2d68e8657b295'},
 {'id': '18bb73bcff9061b6', 'threadId': '18bb73bcff9061b6'},
 {'id': '18bb029a96d1a418', 'threadId': '18bb01d4114ee948'},
 {'id': '18bb01d4114ee948', 'threadId': '18bb01d4114ee948'}]

In [29]:
Gmail.get_message(service, message_id='18f2d783e2039482')

{'id': '18f2d783e2039482',
 'threadId': '18f2d68e8657b295',
 'labelIds': ['IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX'],
 'snippet': 'Hola Mundo! Cnachito feliz Esto es una prueba de template',
 'payload': {'partId': '',
  'mimeType': 'multipart/mixed',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'jorgeang33@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:ab2:4983:0:b0:1f7:43c8:2ab6 with SMTP id h3csp2257791lqk;        Mon, 29 Apr 2024 22:26:46 -0700 (PDT)'},
   {'name': 'X-Received',
    'value': 'by 2002:a05:6808:4297:b0:3c8:2b1f:5270 with SMTP id dq23-20020a056808429700b003c82b1f5270mr12148981oib.20.1714454806425;        Mon, 29 Apr 2024 22:26:46 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1714454806; cv=none;        d=google.com; s=arc-20160816;        b=mAZSFeCg10f2o6ChuZBaUt1VJXtcdIhlEoRjSAJcr6CAMr+/aQzv6FDTigzLaGRe3T         Ev8h7RJC0De7lO7cPGWSUK4VwHgaZwRqrasvKqas8/30M4L1K1JLBhhzJmu1xJwi5rw7         zeY58WsRCoNUzFkyYDK

In [30]:
Gmail.modify_message(service, message_id='18f2d783e2039482', add_labels=['UNREAD'])

{'id': '18f2d783e2039482',
 'threadId': '18f2d68e8657b295',
 'labelIds': ['UNREAD', 'IMPORTANT', 'CATEGORY_PERSONAL', 'INBOX']}

In [21]:
Gmail.delete_message(service, message_id='192d4f2c07c22588', user_id='me')

In [23]:
# Crear el mensaje
draft_message = Gmail.create_message(
    sender='jorgeang33@@gmail.com',
    to='destinatario@example.com',
    subject='Asunto del correo',
    message_text='Este es el contenido del correo que primero fue un borrador.'
)

Gmail.create_draft(service, user_id='me', message_body=draft_message)

{'id': 'r-91776431384130372',
 'message': {'id': '192d6ce990150bea',
  'threadId': '192d6ce990150bea',
  'labelIds': ['DRAFT']}}

In [24]:
Gmail.send_draft(service, user_id='me', draft_id='r-91776431384130372')

{'id': '192d6cece6912c70',
 'threadId': '192d6ce990150bea',
 'labelIds': ['SENT']}

In [25]:
Gmail.list_labels(service)

[{'id': 'CHAT',
  'name': 'CHAT',
  'messageListVisibility': 'hide',
  'labelListVisibility': 'labelHide',
  'type': 'system'},
 {'id': 'SENT', 'name': 'SENT', 'type': 'system'},
 {'id': 'INBOX', 'name': 'INBOX', 'type': 'system'},
 {'id': 'IMPORTANT',
  'name': 'IMPORTANT',
  'messageListVisibility': 'hide',
  'labelListVisibility': 'labelHide',
  'type': 'system'},
 {'id': 'TRASH',
  'name': 'TRASH',
  'messageListVisibility': 'hide',
  'labelListVisibility': 'labelHide',
  'type': 'system'},
 {'id': 'DRAFT', 'name': 'DRAFT', 'type': 'system'},
 {'id': 'SPAM',
  'name': 'SPAM',
  'messageListVisibility': 'hide',
  'labelListVisibility': 'labelHide',
  'type': 'system'},
 {'id': 'CATEGORY_FORUMS',
  'name': 'CATEGORY_FORUMS',
  'messageListVisibility': 'hide',
  'labelListVisibility': 'labelHide',
  'type': 'system'},
 {'id': 'CATEGORY_UPDATES',
  'name': 'CATEGORY_UPDATES',
  'messageListVisibility': 'hide',
  'labelListVisibility': 'labelHide',
  'type': 'system'},
 {'id': 'CATEGORY

In [26]:
Gmail.get_profile(service)

{'emailAddress': 'jorgeang33@gmail.com',
 'messagesTotal': 9336,
 'threadsTotal': 8850,
 'historyId': '1816849'}